In [1]:
import json
f = open('../parent_to_child_class_from_0_to_5.json')
parent_to_child_class_from_0_to_5 = json.load(f)
print(parent_to_child_class_from_0_to_5)

{'11': {'15': 0, '19': 1, '21': 2, '31': 3, '38': 4}, '15': {'27': 0, '29': 1, '44': 2, '78': 3, '93': 4}, '4': {'0': 0, '51': 1, '53': 2, '57': 3, '83': 4}, '14': {'2': 0, '11': 1, '35': 2, '46': 3, '98': 4}, '1': {'1': 0, '32': 1, '67': 2, '73': 3, '91': 4}, '5': {'22': 0, '39': 1, '40': 2, '86': 3, '87': 4}, '18': {'8': 0, '13': 1, '48': 2, '58': 3, '90': 4}, '3': {'9': 0, '10': 1, '16': 2, '28': 3, '61': 4}, '10': {'23': 0, '33': 1, '49': 2, '60': 3, '71': 4}, '17': {'47': 0, '52': 1, '56': 2, '59': 3, '96': 4}, '2': {'54': 0, '62': 1, '70': 2, '82': 3, '92': 4}, '9': {'12': 0, '17': 1, '37': 2, '68': 3, '76': 4}, '8': {'3': 0, '42': 1, '43': 2, '88': 3, '97': 4}, '16': {'36': 0, '50': 1, '65': 2, '74': 3, '80': 4}, '6': {'5': 0, '20': 1, '25': 2, '84': 3, '94': 4}, '12': {'34': 0, '63': 1, '64': 2, '66': 3, '75': 4}, '19': {'41': 0, '69': 1, '81': 2, '85': 3, '89': 4}, '7': {'6': 0, '7': 1, '14': 2, '18': 3, '24': 4}, '13': {'26': 0, '45': 1, '77': 2, '79': 3, '99': 4}, '0': {'4':

In [2]:
from __future__ import print_function
from PIL import Image
import os
import os.path
import errno
import numpy as np
import sys
import torch
if sys.version_info[0] == 2:
    import cPickle as pickle
else:
    import pickle

import torch.utils.data as data
from torchvision.datasets.utils import download_url, check_integrity
import torchvision.transforms as tt
from torchvision import datasets, transforms
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import time
from sklearn.metrics import *
from torchsummary import summary

class CIFAR10(data.Dataset):
    """`CIFAR10 <https://www.cs.toronto.edu/~kriz/cifar.html>`_ Dataset.
    Args:
        root (string): Root directory of dataset where directory
            ``cifar-10-batches-py`` exists.
        train (bool, optional): If True, creates dataset from training set, otherwise
            creates from test set.
        transform (callable, optional): A function/transform that  takes in an PIL image
            and returns a transformed version. E.g, ``transforms.RandomCrop``
        target_transform (callable, optional): A function/transform that takes in the
            target and transforms it.
        download (bool, optional): If true, downloads the dataset from the internet and
            puts it in root directory. If dataset is already downloaded, it is not
            downloaded again.
    """
    base_folder = 'cifar-10-batches-py'
    url = "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"
    filename = "cifar-10-python.tar.gz"
    tgz_md5 = 'c58f30108f718f92721af3b95e74349a'
    train_list = [
        ['data_batch_1', 'c99cafc152244af753f735de768cd75f'],
        ['data_batch_2', 'd4bba439e000b95fd0a9bffe97cbabec'],
        ['data_batch_3', '54ebc095f3ab1f0389bbae665268c751'],
        ['data_batch_4', '634d18415352ddfa80567beed471001a'],
        ['data_batch_5', '482c414d41f54cd18b22e5b47cb7c3cb'],
    ]

    test_list = [
        ['test_batch', '40351d587109b95175f43aff81a1287e'],
    ]

    meta = {
        "filename": "batches.meta",
        "key": "label_names",
        "md5": "5ff9c542aee3614f3951f8cda6e48888",
    }


    def __init__(self, root, train=True,
                 transform=None, target_transform=None,
                 download=False, coarse=False):
        self.root = os.path.expanduser(root)
        self.transform = transform
        self.target_transform = target_transform
        self.train = train  # training set or test set
        self.coarse = coarse

        if download:
            self.download()

        if not self._check_integrity():
            raise RuntimeError('Dataset not found or corrupted.' +
                               ' You can use download=True to download it')

        # now load the picked numpy arrays
        if self.train:
            self.train_data = []
            self.train_labels = []
            self.train_coarse_labels = []
            for fentry in self.train_list:
                f = fentry[0]
                file = os.path.join(self.root, self.base_folder, f)
                fo = open(file, 'rb')
                if sys.version_info[0] == 2:
                    entry = pickle.load(fo)
                else:
                    entry = pickle.load(fo, encoding='latin1')
                self.train_data.append(entry['data'])
                if 'labels' in entry:
                    self.train_labels += entry['labels']
                else:
                    self.train_labels += entry['fine_labels']
                    if self.coarse:
                        self.train_coarse_labels += entry['coarse_labels']
                fo.close()

            self.train_data = np.concatenate(self.train_data)
            self.train_data = self.train_data.reshape((len(self.train_data), 3, 32, 32))
            self.train_data = self.train_data.transpose((0, 2, 3, 1))  # convert to HWC
        else:
            f = self.test_list[0][0]
            file = os.path.join(self.root, self.base_folder, f)
            fo = open(file, 'rb')
            if sys.version_info[0] == 2:
                entry = pickle.load(fo)
            else:
                entry = pickle.load(fo, encoding='latin1')
            self.test_data = entry['data']

            if 'labels' in entry:
                self.test_labels = entry['labels']
            else:
                self.test_labels = entry['fine_labels']
                if self.coarse:
                    self.test_coarse_labels = entry['coarse_labels']
            fo.close()
            self.test_data = self.test_data.reshape((len(self.test_data), 3, 32, 32))
            self.test_data = self.test_data.transpose((0, 2, 3, 1))  # convert to HWC

    def __getitem__(self, index):
        """
        Args:
            index (int): Index
        Returns:
            tuple: (image, target) where target is index of the target class.
        """
        if self.train:
            img, target = self.train_data[index], self.train_labels[index]
            if self.coarse:
                coarse_target = self.train_coarse_labels[index]
        else:
            img, target = self.test_data[index], self.test_labels[index]
            if self.coarse:
                coarse_target = self.test_coarse_labels[index]

        # doing this so that it is consistent with all other datasets
        # to return a PIL Image
        img = Image.fromarray(img)

        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)
        if not self.coarse:
            return img, target
        else:
            return img, target, coarse_target

    def __len__(self):
        if self.train:
            return len(self.train_data)
        else:
            return len(self.test_data)

    def _check_integrity(self):
        root = self.root
        for fentry in (self.train_list + self.test_list):
            filename, md5 = fentry[0], fentry[1]
            fpath = os.path.join(root, self.base_folder, filename)
            if not check_integrity(fpath, md5):
                return False
        return True

    def download(self):
        import tarfile

        if self._check_integrity():
            print('Files already downloaded and verified')
            return

        root = self.root
        download_url(self.url, root, self.filename, self.tgz_md5)

        # extract file
        cwd = os.getcwd()
        tar = tarfile.open(os.path.join(root, self.filename), "r:gz")
        os.chdir(root)
        tar.extractall()
        tar.close()
        os.chdir(cwd)


class CIFAR100(CIFAR10):
    """`CIFAR100 <https://www.cs.toronto.edu/~kriz/cifar.html>`_ Dataset.
    This is a subclass of the `CIFAR10` Dataset.
    """
    base_folder = 'cifar-100-python'
    url = "https://www.cs.toronto.edu/~kriz/cifar-100-python.tar.gz"
    filename = "cifar-100-python.tar.gz"
    tgz_md5 = 'eb9058c3a382ffc7106e4002c42a8d85'
    train_list = [
        ['train', '16019d7e3df5f24257cddd939b257f8d'],
    ]

    test_list = [
        ['test', 'f0ef6b0ae62326f3e7ffdfab6717acfc'],
    ]


In [3]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [4]:
batch_size = 400
epochs = 120
max_lr = 0.001
grad_clip = 0.01
weight_decay =0.001
opt_func = torch.optim.Adam

In [5]:
transform_train = tt.Compose([tt.RandomCrop(32, padding=4,padding_mode='reflect'), 
                         tt.RandomHorizontalFlip(), 
                         tt.ToTensor(), 
                         #tt.Normalize(mean,std,inplace=True) 
                         ]
                         )
transform_test = tt.Compose([tt.ToTensor(), 
                             #tt.Normalize(mean,std)
                             ])

train_data = CIFAR100('./data', train=True,
                 transform=transform_train,
                 download=True, coarse=True)
test_data = CIFAR100('./data', train=False,
                 transform=transform_test,
                 download=True, coarse=True)

train_length = train_data.__len__() # Length training dataset
train_indices = np.arange(train_length)
    
train_loader = torch.utils.data.DataLoader(
                        train_data, 
                        batch_size=batch_size, 
                        num_workers=2,
                        pin_memory=True,
                    )

test_loader = torch.utils.data.DataLoader(
                        test_data, 
                        batch_size=batch_size*2,
                        num_workers=2,
                        pin_memory=True
                    )

Files already downloaded and verified
Files already downloaded and verified


In [6]:
device = get_default_device()
trainloader = DeviceDataLoader(train_loader, device)
testloader = DeviceDataLoader(test_loader, device)

In [7]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

class ImageClassificationBase(nn.Module):
    def __init__(self, fine):
        super(ImageClassificationBase, self).__init__()
        self.fine = fine
    
    def training_step(self, batch):
        images, fine, coarse = batch 
        # if self.fine:
        #     labels=fine
        # else:
        #     labels=coarse
        out_parent, out_child = self(images)                  # Generate predictions
        loss_parent = F.cross_entropy(out_parent, coarse)
        loss_child = F.cross_entropy(out_child, fine) # Calculate loss
        return loss_parent+1.5*loss_child
    
    def validation_step(self, batch):
        images, fine, coarse = batch 
        # if self.fine:
        #     labels=fine
        # else:
        #     labels=coarse
        #out = self(images)                    # Generate predictions
        #loss = F.cross_entropy(out, labels)
        out_parent, out_child = self(images)                  # Generate predictions
        loss_parent = F.cross_entropy(out_parent, coarse)
        loss_child = F.cross_entropy(out_child, fine)   # Calculate loss
        acc_parent = accuracy(out_parent, coarse)
        acc_child = accuracy(out_child, fine)           # Calculate accuracy
        return {'val_loss_parent': loss_parent.detach(), 'val_acc_parent': acc_parent, 'val_loss_child':loss_child.detach(), 'val_acc_child': acc_child}
        
    def validation_epoch_end(self, outputs):
        batch_losses_parent = [x['val_loss_parent'] for x in outputs]
        epoch_loss_parent = torch.stack(batch_losses_parent).mean()   # Combine losses
        #
        batch_accs_parent = [x['val_acc_parent'] for x in outputs]
        epoch_acc_parent = torch.stack(batch_accs_parent).mean()  # Combine accuracies
        ###     
        batch_losses_child = [x['val_loss_child'] for x in outputs]
        epoch_loss_child = torch.stack(batch_losses_child).mean()   # Combine losses
        #
        batch_accs_child = [x['val_acc_child'] for x in outputs]
        epoch_acc_child = torch.stack(batch_accs_child).mean() 
        return {'val_loss_parent': epoch_loss_parent.item(), 'val_acc_parent': epoch_acc_parent.item(), 'val_loss_child':epoch_loss_child.item(), 
                'val_acc_child':epoch_acc_child.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], last_lr: {:.5f}, train_loss: {:.4f}, val_loss_parent: {:.4f}, val_acc_parent: {:.4f}, val_loss_child: {:.4f}, val_acc_child: {:.4f}".format(
            epoch, result['lrs'][-1], result['train_loss'], result['val_loss_parent'], result['val_acc_parent'], result['val_loss_child'], result['val_acc_child']))
        
def conv_block(in_channels, out_channels, pool=False):
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1), 
              nn.BatchNorm2d(out_channels), 
              nn.ReLU(inplace=True)]
    if pool: layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)

class ResNet9(ImageClassificationBase):
    def __init__(self, in_channels, num_classes, fine):
        super().__init__(fine)
        
        self.conv1 = conv_block(in_channels, 64)
        self.conv2 = conv_block(64, 128, pool=True) 
        self.res1 = nn.Sequential(conv_block(128, 128), conv_block(128, 128)) 
        
        self.conv3 = conv_block(128, 256, pool=True)
        self.conv4 = conv_block(256, 512, pool=True) 
        self.res2 = nn.Sequential(conv_block(512, 512), conv_block(512, 512)) 
        self.conv5 = conv_block(512, 1028, pool=True) 
        self.res3 = nn.Sequential(conv_block(1028, 1028), conv_block(1028, 1028))

        #-------------------
        self.classifier_parent = nn.Sequential(nn.MaxPool2d(2), # 1028 x 1 x 1
                                        nn.Flatten(), # 1028 
                                        nn.Linear(1028, 20))
                                 # 1028 -> 100 

        self.classifier_child = nn.Sequential(self.classifier_parent,
                                nn.ReLU(),
                                nn.Linear(20, num_classes)
                                ) # 1028 -> 100 
        
    def forward(self, xb):
        out = self.conv1(xb)
        out = self.conv2(out)
        out = self.res1(out) + out
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.res2(out) + out
        out = self.conv5(out)
        out = self.res3(out) + out
        parent_out=self.classifier_parent(out)
        #child_out = self.classifier_child(out)
        return parent_out

model100 = to_device(ResNet9(3, 100, True), device)

In [8]:
@torch.no_grad()
def evaluate(model, test_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in test_loader]
    return model.validation_epoch_end(outputs)

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def fit_one_cycle(epochs, max_lr, model, train_loader, test_loader, 
                  weight_decay=0, grad_clip=None, opt_func=torch.optim.SGD):
    torch.cuda.empty_cache()
    history = []
    
    # Set up cutom optimizer with weight decay
    optimizer = opt_func(model.parameters(), max_lr, weight_decay=weight_decay)
    # Set up one-cycle learning rate scheduler
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs, 
                                                steps_per_epoch=len(train_loader))
    
    for epoch in range(epochs):
        # Training Phase 
        model.train()
        train_losses = []
        lrs = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            
            # Gradient clipping
            if grad_clip: 
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)
            
            optimizer.step()
            optimizer.zero_grad()
            
            # Record & update learning rate
            lrs.append(get_lr(optimizer))
            sched.step()
        
        # Validation phase
        result = evaluate(model, test_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        result['lrs'] = lrs
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [9]:
# Generate testing accuracy, predicted label, confusion matrix, and table for classification report
def test_label_predictions(model, device, test_loader, fine):
    model.eval()
    actuals = []
    predictions = []
    with torch.no_grad():
        if fine:
            for data, target, _ in test_loader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                prediction = output.argmax(dim=1, keepdim=True)
                print("predicted ", prediction, " actual: ", target)
                actuals.extend(target.view_as(prediction))
                predictions.extend(prediction)
        else:
            for data, _, target in test_loader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                prediction = output.argmax(dim=1, keepdim=True)
                print("predicted ", prediction, " actual: ", target)
                actuals.extend(target.view_as(prediction))
                predictions.extend(prediction)

    return [i.item() for i in actuals], [i.item() for i in predictions]

In [10]:
modelX_to_parent = ResNet9(3, 100, True)
# modelX_to_parent.classifier_child = nn.Sequential(modelX_to_parent.classifier_parent,
#                                         nn.ReLU(),
#                                         nn.Linear(20, 5)
#                                         )
modelX_to_parent.load_state_dict(torch.load('../group_1028_to_parent_Loss_parent_and_child_pretrained_model.h5'))
for param in modelX_to_parent.parameters():
    param.requires_grad=False

In [11]:
modelX_to_parent=to_device(modelX_to_parent, device)

In [12]:
class SimpleModel(nn.Module):

    def __init__(self):
        super(SimpleModel, self).__init__()

        self.relu = nn.ReLU()
        self.fc0 = nn.Linear(20, 5)
        self.fc1 = nn.Linear(20, 5)
        self.fc2 = nn.Linear(20, 5)
        self.fc3 = nn.Linear(20, 5)
        self.fc4 = nn.Linear(20, 5)
        self.fc5 = nn.Linear(20, 5)
        self.fc6 = nn.Linear(20, 5)
        self.fc7 = nn.Linear(20, 5)
        self.fc8 = nn.Linear(20, 5)
        self.fc9 = nn.Linear(20, 5)
        self.fc10 = nn.Linear(20, 5)
        self.fc11 = nn.Linear(20, 5)
        self.fc12 = nn.Linear(20, 5)
        self.fc13 = nn.Linear(20, 5)
        self.fc14 = nn.Linear(20, 5)
        self.fc15 = nn.Linear(20, 5)
        self.fc16 = nn.Linear(20, 5)
        self.fc17 = nn.Linear(20, 5)
        self.fc18 = nn.Linear(20, 5)
        self.fc19 = nn.Linear(20, 5)

        self.dict={0:self.fc0,
                   1:self.fc1,
                   2:self.fc2,
                   3:self.fc3,
                   4:self.fc4,
                   5:self.fc5,
                   6:self.fc6,
                   7:self.fc7,
                   8:self.fc8,
                   9:self.fc9,
                   10:self.fc1,
                   11:self.fc11,
                   12:self.fc12,
                   13:self.fc13,
                   14:self.fc14,
                   15:self.fc15,
                   16:self.fc16,
                   17:self.fc17,
                   18:self.fc18,
                   19:self.fc19
                   }

    def forward(self, x):
        x = self.relu(x)
        y=torch.zeros(x.shape[0],1)
        for i in range(x.shape[0]):
            parent=x[i].argmax(dim=0).item()
            child=self.dict[parent](x[i]).argmax(dim=0).item()
            y[i]=torch.tensor(int(list(parent_to_child_class_from_0_to_5[str(parent)].keys())[list(parent_to_child_class_from_0_to_5[str(parent)].values()).index(child)]))

        return y

In [13]:
model = SimpleModel()

In [14]:
model=to_device(model, device)

In [15]:
with torch.no_grad():
    model.fc0.weight=nn.Parameter(torch.load("group_1028_to_parent_Loss_parent_and_child_pretrained_model_For_parent_{}_weight_reduced_size.pt".format(0)))
    model.fc0.bias=nn.Parameter(torch.load("group_1028_to_parent_Loss_parent_and_child_pretrained_model_For_parent_{}_bias_reduced_size.pt".format(0)))
    model.fc1.weight=nn.Parameter(torch.load("group_1028_to_parent_Loss_parent_and_child_pretrained_model_For_parent_{}_weight_reduced_size.pt".format(1)))
    model.fc1.bias=nn.Parameter(torch.load("group_1028_to_parent_Loss_parent_and_child_pretrained_model_For_parent_{}_bias_reduced_size.pt".format(1)))
    model.fc2.weight=nn.Parameter(torch.load("group_1028_to_parent_Loss_parent_and_child_pretrained_model_For_parent_{}_weight_reduced_size.pt".format(2)))
    model.fc2.bias=nn.Parameter(torch.load("group_1028_to_parent_Loss_parent_and_child_pretrained_model_For_parent_{}_bias_reduced_size.pt".format(2)))
    model.fc3.weight=nn.Parameter(torch.load("group_1028_to_parent_Loss_parent_and_child_pretrained_model_For_parent_{}_weight_reduced_size.pt".format(3)))
    model.fc3.bias=nn.Parameter(torch.load("group_1028_to_parent_Loss_parent_and_child_pretrained_model_For_parent_{}_bias_reduced_size.pt".format(3)))
    model.fc4.weight=nn.Parameter(torch.load("group_1028_to_parent_Loss_parent_and_child_pretrained_model_For_parent_{}_weight_reduced_size.pt".format(4)))
    model.fc4.bias=nn.Parameter(torch.load("group_1028_to_parent_Loss_parent_and_child_pretrained_model_For_parent_{}_bias_reduced_size.pt".format(4)))

    model.fc5.weight=nn.Parameter(torch.load("group_1028_to_parent_Loss_parent_and_child_pretrained_model_For_parent_{}_weight_reduced_size.pt".format(5)))
    model.fc5.bias=nn.Parameter(torch.load("group_1028_to_parent_Loss_parent_and_child_pretrained_model_For_parent_{}_bias_reduced_size.pt".format(5)))
    model.fc6.weight=nn.Parameter(torch.load("group_1028_to_parent_Loss_parent_and_child_pretrained_model_For_parent_{}_weight_reduced_size.pt".format(6)))
    model.fc6.bias=nn.Parameter(torch.load("group_1028_to_parent_Loss_parent_and_child_pretrained_model_For_parent_{}_bias_reduced_size.pt".format(6)))
    model.fc7.weight=nn.Parameter(torch.load("group_1028_to_parent_Loss_parent_and_child_pretrained_model_For_parent_{}_weight_reduced_size.pt".format(7)))
    model.fc7.bias=nn.Parameter(torch.load("group_1028_to_parent_Loss_parent_and_child_pretrained_model_For_parent_{}_bias_reduced_size.pt".format(7)))
    model.fc8.weight=nn.Parameter(torch.load("group_1028_to_parent_Loss_parent_and_child_pretrained_model_For_parent_{}_weight_reduced_size.pt".format(8)))
    model.fc8.bias=nn.Parameter(torch.load("group_1028_to_parent_Loss_parent_and_child_pretrained_model_For_parent_{}_bias_reduced_size.pt".format(8)))
    model.fc9.weight=nn.Parameter(torch.load("group_1028_to_parent_Loss_parent_and_child_pretrained_model_For_parent_{}_weight_reduced_size.pt".format(9)))
    model.fc9.bias=nn.Parameter(torch.load("group_1028_to_parent_Loss_parent_and_child_pretrained_model_For_parent_{}_bias_reduced_size.pt".format(9)))

    model.fc10.weight=nn.Parameter(torch.load("group_1028_to_parent_Loss_parent_and_child_pretrained_model_For_parent_{}_weight_reduced_size.pt".format(10)))
    model.fc10.bias=nn.Parameter(torch.load("group_1028_to_parent_Loss_parent_and_child_pretrained_model_For_parent_{}_bias_reduced_size.pt".format(10)))
    model.fc11.weight=nn.Parameter(torch.load("group_1028_to_parent_Loss_parent_and_child_pretrained_model_For_parent_{}_weight_reduced_size.pt".format(11)))
    model.fc11.bias=nn.Parameter(torch.load("group_1028_to_parent_Loss_parent_and_child_pretrained_model_For_parent_{}_bias_reduced_size.pt".format(11)))
    model.fc12.weight=nn.Parameter(torch.load("group_1028_to_parent_Loss_parent_and_child_pretrained_model_For_parent_{}_weight_reduced_size.pt".format(12)))
    model.fc12.bias=nn.Parameter(torch.load("group_1028_to_parent_Loss_parent_and_child_pretrained_model_For_parent_{}_bias_reduced_size.pt".format(12)))
    model.fc13.weight=nn.Parameter(torch.load("group_1028_to_parent_Loss_parent_and_child_pretrained_model_For_parent_{}_weight_reduced_size.pt".format(13)))
    model.fc13.bias=nn.Parameter(torch.load("group_1028_to_parent_Loss_parent_and_child_pretrained_model_For_parent_{}_bias_reduced_size.pt".format(13)))
    model.fc14.weight=nn.Parameter(torch.load("group_1028_to_parent_Loss_parent_and_child_pretrained_model_For_parent_{}_weight_reduced_size.pt".format(14)))
    model.fc14.bias=nn.Parameter(torch.load("group_1028_to_parent_Loss_parent_and_child_pretrained_model_For_parent_{}_bias_reduced_size.pt".format(14)))

    model.fc15.weight=nn.Parameter(torch.load("group_1028_to_parent_Loss_parent_and_child_pretrained_model_For_parent_{}_weight_reduced_size.pt".format(15)))
    model.fc15.bias=nn.Parameter(torch.load("group_1028_to_parent_Loss_parent_and_child_pretrained_model_For_parent_{}_bias_reduced_size.pt".format(15)))
    model.fc16.weight=nn.Parameter(torch.load("group_1028_to_parent_Loss_parent_and_child_pretrained_model_For_parent_{}_weight_reduced_size.pt".format(16)))
    model.fc16.bias=nn.Parameter(torch.load("group_1028_to_parent_Loss_parent_and_child_pretrained_model_For_parent_{}_bias_reduced_size.pt".format(16)))
    model.fc17.weight=nn.Parameter(torch.load("group_1028_to_parent_Loss_parent_and_child_pretrained_model_For_parent_{}_weight_reduced_size.pt".format(17)))
    model.fc17.bias=nn.Parameter(torch.load("group_1028_to_parent_Loss_parent_and_child_pretrained_model_For_parent_{}_bias_reduced_size.pt".format(17)))
    model.fc18.weight=nn.Parameter(torch.load("group_1028_to_parent_Loss_parent_and_child_pretrained_model_For_parent_{}_weight_reduced_size.pt".format(18)))
    model.fc18.bias=nn.Parameter(torch.load("group_1028_to_parent_Loss_parent_and_child_pretrained_model_For_parent_{}_bias_reduced_size.pt".format(18)))
    model.fc19.weight=nn.Parameter(torch.load("group_1028_to_parent_Loss_parent_and_child_pretrained_model_For_parent_{}_weight_reduced_size.pt".format(19)))
    model.fc19.bias=nn.Parameter(torch.load("group_1028_to_parent_Loss_parent_and_child_pretrained_model_For_parent_{}_bias_reduced_size.pt".format(19)))

In [34]:
for param in model.parameters():
    param.requires_grad=True

In [17]:
from ptflops import get_model_complexity_info

In [24]:
macs_classifier, params_classifier = get_model_complexity_info(modelX_to_parent.classifier_parent, (1028,2,2), as_strings=False, print_per_layer_stat=False, verbose=False) 
print(macs_classifier)
print(params_classifier)

28804.0
0


In [39]:
macs_classifier, params_classifier = get_model_complexity_info(model.fc0, (1,1,20), as_strings=False, print_per_layer_stat=False, verbose=False) 
print(macs_classifier)
print(params_classifier)

105.0
105


In [40]:
# MACs for parent and child summed up
print(28804.0+105.0)

28909.0


In [63]:
modelX_to_parent.eval()
model.eval()
actuals = []
predictions = []
with torch.no_grad():
    for data, target, _ in testloader:
        data, target = data.to(device), target.to(device)
        output = modelX_to_parent(data)
        output_child=model(output)
        prediction=output_child
        actuals.extend(target.view_as(prediction))
        predictions.extend(prediction)

y_test, y_pred= [i.item() for i in actuals], [i.item() for i in predictions]

In [64]:
print('Accuracy score: %f' % accuracy_score(y_test, y_pred))

Accuracy score: 0.635500


In [174]:
confusion_matrix(y_test, y_pred)

array([[95,  0,  0, ...,  0,  0,  0],
       [ 0, 84,  0, ...,  0,  0,  0],
       [ 1,  1, 55, ...,  0,  1,  0],
       ...,
       [ 0,  0,  0, ..., 69,  1,  1],
       [ 1,  0,  1, ...,  0, 29,  0],
       [ 0,  0,  0, ...,  0,  0, 73]])

In [176]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.75      0.95      0.84       100
           1       0.84      0.84      0.84       100
           2       0.71      0.55      0.62       100
           3       0.71      0.42      0.53       100
           4       0.50      0.56      0.53       100
           5       0.55      0.78      0.65       100
           6       0.56      0.81      0.66       100
           7       0.64      0.73      0.68       100
           8       0.89      0.78      0.83       100
           9       0.82      0.81      0.81       100
          10       0.70      0.32      0.44       100
          11       0.45      0.54      0.49       100
          12       0.74      0.71      0.72       100
          13       0.57      0.69      0.62       100
          14       0.71      0.60      0.65       100
          15       0.60      0.73      0.66       100
          16       0.64      0.63      0.63       100
          17       0.79    

/home/arsen.abzhanov/.conda/envs/Copy_of_CV701_env_for_Temporal_Santosh/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/arsen.abzhanov/.conda/envs/Copy_of_CV701_env_for_Temporal_Santosh/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/arsen.abzhanov/.conda/envs/Copy_of_CV701_env_for_Temporal_Santosh/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samp

In [88]:
parent_to_child_class_from_0_to_5['11']
# i will use argmax and pass here instead of 11

{'15': 0, '19': 1, '21': 2, '31': 3, '38': 4}

In [92]:
res=11
parent_dict=parent_to_child_class_from_0_to_5[str(res)]
print(parent_dict)

{'15': 0, '19': 1, '21': 2, '31': 3, '38': 4}


In [90]:
list(parent_to_child_class_from_0_to_5['11'].keys())[list(parent_to_child_class_from_0_to_5['11'].values()).index(4)]
# i will use argmax and pass here instead of 4

'38'

In [93]:
list(parent_dict.keys())[list(parent_dict.values()).index(4)]

'38'